In [5]:
!pip install opendatasets

In [2]:
!pip install radiant_mlhub -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.1/127.1 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.4/137.4 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 11.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is t

In [4]:
!pip install geopandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 22.4 MB/s eta 0:00:00
  Attempting uninstall: shapely
    Found existing installation: Shapely 1.8.5.post1
    Uninstalling Shapely-1.8.5.post1:
      Successfully uninstalled Shapely-1.8.5.post1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
radiant-mlhub 0.5.5 requires shapely~=1.8.0, but you have shapely 2.0.7 which is incompatible.


ERROR: Could not find a version that satisfies the requirement kaggle_secrets (from versions: none)
ERROR: No matching distribution found for kaggle_secrets


In [3]:
import os
from radiant_mlhub import get_session
from radiant_mlhub import Dataset,Collection
import shutil
import geopandas as gpd

#get key
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("RadiantEarth_MLHub")


#authenticate
os.environ['MLHUB_API_KEY'] = secret_value_0
session = get_session()

OSError: Could not find kaggle.json. Make sure it's located in /root/.config/kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/

# Imports

In [2]:
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 75.7 MB/s eta 0:00:00


In [3]:
import json
import tensorflow as tf
import tensorflow.keras.backend as K
import numpy as np
import pandas as pd
import seaborn as sns
import os
import gc
import rasterio as rio
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib import  cm
import cv2
from matplotlib import animation
from IPython.display import HTML
from tqdm import tqdm
from sklearn.model_selection import train_test_split

# Config

In [4]:
class CFG:
    seed = 7
    img_size = (256,256)
    BATCH_SIZE = 3
    Autotune = tf.data.AUTOTUNE
    validation_size = 0.2
    class_dict= {0:'No Flooding',
                 1: 'Flooding'}

    test_run = False



# Input data

    Read more about the dataset here : https://clmrmb.github.io/SEN12-FLOOD/

In [ ]:
s1_labels = '../input/sen12flood/sen12flood/sen12floods_s1_labels/sen12floods_s1_labels/'
s1_tiles = '../input/sen12flood/sen12flood/sen12floods_s1_source/sen12floods_s1_source/'

s2_tiles = '../input/sen12flood/sen12flood/sen12floods_s2_source/sen12floods_s2_source/'
s2_labels = '../input/sen12flood/sen12flood/sen12floods_s2_labels/sen12floods_s2_labels/'


s1_check = 0
for file in os.listdir(s1_labels):
    if os.path.exists(s1_tiles + '/' + file.replace('labels','source')):
        s1_check += 1


assert s1_check == len(os.listdir(s1_tiles)), 'You my friend , are definintely a idiot!'

s2_check = 0
for file in os.listdir(s2_labels):
    if os.path.exists(s2_tiles + '/' + file.replace('labels','source')):
        s2_check += 1


assert s2_check == len(os.listdir(s2_tiles)), 'You my friend , are definintely  the idiot!'


s1_check,s2_check

# Make a dataset of paths and labels

# Helper Functions

In [ ]:
def load_json(path):
    '''loads a json file'''
    with open(path,'r') as file:
        js = json.load(file)

    return js

# collectionss1 = load_json('../input/sen12flood/sen12flood/sen12floods_s1_source/sen12floods_s1_source/collection.json')
# collections2= load_json('../input/sen12flood/sen12flood/sen12floods_s2_source/sen12floods_s2_source/collection.json')
# collections2

In [ ]:
def process_label_json(label_json):
    '''process a single label json'''
    info_dict = {}

    info_dict['geometry'] = label_json['geometry']['coordinates']
    info_dict['label'] = label_json['properties']['FLOODING']
    info_dict['date'] = label_json['properties']['date']
    info_dict['tile_number'] = label_json['properties']['tile']
#     info_dict['full_data_coverage']= label_json['properties']['FULL-DATA-COVERAGE']

    return info_dict


def process_label_stac(stac_json):
    return stac_json['id']




def image_path_from_label_dir(image_parent_dir:str,
                              label_file :str)->str:

    return image_parent_dir + '/' + label_file.replace('labels','source')



def process_json(label_path,image_directory):
    '''get the data for a single example
     Inputs
     label_path : path to the label folder
     image_directory: path to the corresponding image directory'''



    #get image directory for that label
    folder_id = label_path.rsplit('/',1)[1]
    image_dir_path = image_path_from_label_dir(image_directory,folder_id)

    if not os.path.exists(image_dir_path):
        return {'File_not_found':image_dir_path}


    for file in os.listdir(label_path):
        #if image dir exists
        if file.startswith('labels'):
            label_json = load_json(os.path.join(label_path,file))
        else:
            stac_json = load_json(os.path.join(label_path,file))


    #get data
    info_dict = process_label_json(label_json)

    #get id
    info_dict['id'] = process_label_stac(stac_json)

    #location id
    info_dict['location_id'] = info_dict['id'].split('_')[3]


    info_dict['image_dir'] = image_dir_path


    return info_dict


In [ ]:
def get_dataframe(label_directory,image_directory):
    '''get dataframe from the nested label directory'''
    records = []


    for folder in os.listdir(label_directory):
        if folder.startswith('sen12'):
#             print(folder,label_directory)
            folder_path = label_directory + '/' + folder


            #get data for a single example
            feature = process_json(label_path=folder_path,
                                   image_directory=image_directory)


            records.append(feature)


    return pd.DataFrame.from_records(data = records)



def type_cast_dataset(dataset):
    '''typecasting columns in dataset'''
    dataset['label'] = dataset['label'].astype(int)

    dataset['date'] = pd.to_datetime(dataset['date'])
    dataset['tile_number'] = dataset['tile_number'].astype('int8')


    return dataset


In [ ]:
%%time
s1_data = type_cast_dataset(
                            get_dataframe(
                                label_directory=s1_labels,
                                image_directory=s1_tiles
                                        )
                            )


s2_data = type_cast_dataset(
                            get_dataframe(label_directory=s2_labels,
                                          image_directory=s2_tiles)
                            )

print(f'Number of unique locations in Sentinel1 (SAR) data : {s1_data.location_id.nunique()}')
print(f'Number of unique locations in Sentinel2 (optical) data : {s2_data.location_id.nunique()}')

s1_data.shape,s2_data.shape

In [ ]:
# saving datasets
s1_data.to_csv('s1_data.csv',index=False)
s2_data.to_csv('s2_data.csv',index=False)

In [ ]:
def load_raster(filepath):
    '''load a single band raster'''
    with rio.open(filepath) as file:
        raster = file.read().squeeze(axis=0)

    return raster



**Loading multiple raster bands as single raster**

In [ ]:
def load_s1_tiffs(folder,
                  scaling_values=[50.,100.]):
    images = []
    i = 0
    for im in sorted(os.listdir(folder)):

        if im.rsplit('.',maxsplit=1)[1] == 'tif':

            path = folder + '/' + im
            band = load_raster(path)
            band = band / scaling_values[i]

            band = cv2.resize(band,
                              CFG.img_size)

            images.append(band)
            i+=1

    return np.dstack(images)


def load_s2_tiffs(folder,
                  scaling_value=10000.):
    images = []
    for im in sorted(os.listdir(folder)):
        if im.rsplit('.',maxsplit=1)[1] == 'tif':
            path = folder + '/' + im
            band = load_raster(path)
            band = band/ scaling_value

            band = cv2.resize(band,CFG.img_size)
            images.append(band)

    return np.dstack(images)

def load_rgb_tiffs(folder,
                  scaling_value=10000.):
    '''load R,G and B bands'''

    images = []
    for im in sorted(os.listdir(folder)):
        name,file_format = im.rsplit('.',maxsplit=1)
        if ((file_format== 'tif') and (name in ['B02','B03','B04'])):
            path = folder + '/' + im
            band = load_raster(path)
            band = band/ scaling_value

            band = cv2.resize(band,CFG.img_size)
            images.append(band)

    return np.dstack(images)[:,:,::-1]



def tf_load_s1(path):
    path = path.numpy().decode('utf-8')
    return load_s1_tiffs(path)



def tf_load_s2(path):
    path = path.numpy().decode('utf-8')
    return load_s2_tiffs(path)


def tf_load_rgb(path):
    path = path.numpy().decode('utf-8')
    return load_rgb_tiffs(path)

def process_image_s1(filename):
    '''function for preprocessing in tensorflow data'''

    return tf.py_function(tf_load_s1,
                          [filename],
                          tf.float32)



def process_image_s2(filename):
    '''function for preprocessing in tensorflow data'''

    return tf.py_function(tf_load_s2,
                          [filename],
                          tf.float32)



def process_image_rgb(filename):
    '''function for preprocessing in tensorflow data'''

    return tf.py_function(tf_load_rgb,
                          [filename],
                          tf.float32)


In [ ]:
def count_rasters_in_folder(path):
    count = 0

    for file in os.listdir(path):
        if file.rsplit('.',1)[1] == 'tif':
            count +=1

    return count


s2_data['raster_count'] = s2_data.image_dir.apply(lambda x : count_rasters_in_folder(x))

#value counts
s2_data['raster_count'].value_counts()


s2_data=s2_data[s2_data['raster_count']==12] # take only valid rasters
# s2_data[s2_data['raster_count']==0]['location_id'].value_counts()

# Visualize some images

**lets take a look at some optical RGB images**

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(16,8))

plt.subplot(1,2,1)
chk = load_s2_tiffs(s2_data.query('label==1')['image_dir'].values[11])
plt.imshow(chk[:,:,1:4][:,:,::-1])
plt.title('Flooding')
plt.axis('off')


chk = load_s2_tiffs(s2_data.query('label==0')['image_dir'].values[20])
plt.subplot(1,2,2)
plt.imshow(chk[:,:,1:4][:,:,::-1])
plt.title('No Flooding')
plt.axis('off')
plt.show()

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(16,8))

plt.subplot(1,2,1)
chk = load_s2_tiffs(s2_data.query('label==1')['image_dir'].values[101])
plt.imshow(chk[:,:,1:4][:,:,::-1])
plt.title('Flooding')
plt.axis('off')


chk = load_s2_tiffs(s2_data.query('label==0')['image_dir'].values[11])
plt.subplot(1,2,2)
plt.imshow(chk[:,:,1:4][:,:,::-1])
plt.title('No Flooding')
plt.axis('off')
plt.show()

**Some SAR images**

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(16,8))

plt.suptitle('SAR images (VH)')

plt.subplot(1,2,1)


chk = load_s1_tiffs(s1_data.query('label==1')['image_dir'].values[11])
plt.imshow(chk[:,:,1])
plt.title('Flooding')
plt.axis('off')


chk = load_s2_tiffs(s2_data.query('label==0')['image_dir'].values[11])
plt.subplot(1,2,2)
plt.imshow(chk[:,:,1])
plt.title('No Flooding')
plt.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
fig,ax = plt.subplots(1,2,figsize=(16,8))
plt.suptitle('SAR images (VV)')


r_idx = np.random.randint(low=0,high=500)
plt.subplot(1,2,1)
chk = load_s1_tiffs(s1_data.query('label==1')['image_dir'].values[r_idx])
plt.imshow(chk[:,:,0])
plt.title('Flooding')
plt.axis('off')


chk = load_s2_tiffs(s2_data.query('label==0')['image_dir'].values[r_idx])
plt.subplot(1,2,2)
plt.imshow(chk[:,:,0])
plt.title('No Flooding')
plt.axis('off')
plt.tight_layout()

plt.show()

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(16,8))
plt.suptitle('SAR images (VV)')


r_idx = np.random.randint(low=0,high=500)
plt.subplot(1,2,1)
chk = load_s1_tiffs(s1_data.query('label==1')['image_dir'].values[r_idx])
plt.imshow(chk[:,:,0])
plt.title('Flooding')
plt.axis('off')


chk = load_s2_tiffs(s2_data.query('label==0')['image_dir'].values[r_idx])
plt.subplot(1,2,2)
plt.imshow(chk[:,:,0])
plt.title('No Flooding')
plt.axis('off')
plt.tight_layout()

plt.show()

**Lets look at the distribution of target values**

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(16,8))

plt.subplot(1,2,1)
sns.countplot(s1_data.label)
plt.title('Sentinel -1 target distribution')


plt.subplot(1,2,2)
sns.countplot(s2_data.label)
plt.title('Sentinel - 2 target distribution')
plt.show()

# Making a TF dataset

    First lets split the dataset into training and validation set. We will stratify based on location id to ensure that locations are well represented in traininng and validation set

In [ ]:
#isolating single loaction ids (as they will be a problem for stratification)

# single example locations
single_index = s2_data['location_id'].value_counts()[s2_data['location_id'].value_counts()==1].index

single_index_df = s2_data[s2_data['location_id'].isin(single_index)].reset_index(drop=True)
s2_data0 = s2_data[~(s2_data['location_id'].isin(single_index))].reset_index(drop=True)

s2_data0.shape,single_index_df.shape

**Split dataset into train and validation splits**

In [ ]:
#traintest split

s1_data_tr,s1_data_val= train_test_split(s1_data,
                                          test_size = CFG.validation_size,
                                          random_state = CFG.seed,
                                          stratify = s1_data.location_id)



s2_data_tr,s2_data_val = train_test_split(s2_data0,
                                          test_size = CFG.validation_size,
                                          random_state = CFG.seed,
                                          stratify =  s2_data0.location_id)

s2_data_tr = s2_data_tr.append(single_index_df,ignore_index=True)

del s2_data0;gc.collect()

In [ ]:
s1_data_tr.label.value_counts(1),s1_data_val.label.value_counts(1)

In [ ]:
s2_data_tr.label.value_counts(1),s2_data_val.label.value_counts(1)

**Function for image augmentations**

    Although the Augmentations are simple, we cannot use them on SAR images , as even simple operations like flipping can change the meaning of the image

In [ ]:
def augment_image_multispectral(image):
    '''perform simple image augmentations'''
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_crop(image, size=(*CFG.img_size,12))

    rot = tf.random.normal((1,),mean = 0.35, stddev=0.15)

    if rot > 0.5:
        image = tf.image.rot90(image)

    return image

def augment_image(image):
    '''perform simple image augmentations'''
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_crop(image, size=(*CFG.img_size,3))

    rot = tf.random.normal((1,),mean = 0.35, stddev=0.15)

    if rot > 0.5:
        image = tf.image.rot90(image)

    return image

In [ ]:
def get_tf_dataset(image_paths,
                   labels=None, # put none for test data set
                   image_processing_fn=None,
                   augment_fn = None
                  ):

    '''returns a tf dataset object
    Inputs:
    image_paths : paths to images
    labels: labels of each image
    image_processing_fn:  function to load and preprocess images
    augment_fn : function to augment images '''

    #seperate datasets
    if labels is not None:
        labels_dataset = tf.data.Dataset.from_tensor_slices(labels)



    image_dataset = tf.data.Dataset.from_tensor_slices(image_paths)
    #load images
    image_dataset = image_dataset.map(image_processing_fn,
                                      num_parallel_calls=tf.data.AUTOTUNE)

    if augment_fn is not None:

        image_dataset = image_dataset.map(augment_fn,
                                          num_parallel_calls=tf.data.AUTOTUNE)


    if labels is not None:
        return tf.data.Dataset.zip((image_dataset,labels_dataset))


    return image_dataset



def optimize_pipeline(tf_dataset,
                      batch_size = CFG.BATCH_SIZE,
                      Autotune_fn = CFG.Autotune,
                      cache= False,
                      batch = True):



    # prefetch(load the data with cpu,while gpu is training) the data in memory
    tf_dataset = tf_dataset.prefetch(buffer_size=Autotune_fn)
    if cache:
        tf_dataset = tf_dataset.cache()                        # store data in RAM

    tf_dataset =  tf_dataset.shuffle(buffer_size=50)         # shuffle

    if batch:
        tf_dataset = tf_dataset.batch(batch_size)              #split the data in batches

    return tf_dataset

**Making dataset pipelines with TF data**

In [ ]:
# Sentinel 1 dataset (not using augmentation here)

S1_dataset_tr = optimize_pipeline(tf_dataset=get_tf_dataset(image_paths = s1_data_tr.image_dir.values,
                                               labels = s1_data_tr.label,
                                               image_processing_fn = process_image_s1),

                                  batch_size = 3 * CFG.BATCH_SIZE)


S1_dataset_val = optimize_pipeline(tf_dataset = get_tf_dataset(image_paths = s1_data_val.image_dir.values,
                                                           labels = s1_data_val.label,
                                                           image_processing_fn = process_image_s1 ),
                                   batch_size = 3* CFG.BATCH_SIZE
                                  )

In [ ]:
#sentinel 2 dataset
S2_dataset_tr = optimize_pipeline(get_tf_dataset(image_paths = s2_data_tr.image_dir.values,
                                                   labels = s2_data_tr.label,
                                                   image_processing_fn = process_image_s2,
                                                   augment_fn = augment_image_multispectral)
                                 )


S2_dataset_val = optimize_pipeline(get_tf_dataset(image_paths = s2_data_val.image_dir.values,
                                                   labels = s2_data_val.label,
                                                   image_processing_fn = process_image_s2,
                                                   augment_fn = augment_image_multispectral)
                                  )




In [ ]:
RGB_dataset_tr = optimize_pipeline(get_tf_dataset(image_paths = s2_data_tr.image_dir.values,
                                                   labels = s2_data_tr.label,
                                                   image_processing_fn = process_image_rgb,
                                                   augment_fn = augment_image),
                                   batch_size = 3* CFG.BATCH_SIZE
                                 )


RGB_dataset_val = optimize_pipeline(get_tf_dataset(image_paths = s2_data_val.image_dir.values,
                                                   labels = s2_data_val.label,
                                                   image_processing_fn = process_image_rgb,
                                                   augment_fn = augment_image),
                                    batch_size = 3* CFG.BATCH_SIZE
                                  )



**Checking the values of pixels in the dataset**

In [ ]:

# max_vals_vv = []
# mean_vals_vv =[]


# max_vals_vh = []
# mean_vals_vh =[]

# for x,y in S1_dataset_tr.as_numpy_iterator():
#     # vv band
#     max_vals_vv.append(x[:,:,:,0].max()); mean_vals_vv.append(x[:,:,:,0].mean())

#     # vh band
#     max_vals_vh.append(x[:,:,:,1].max()); mean_vals_vh.append(x[:,:,:,1].mean())

# # band 1value distributions
# plt.figure(figsize=(16,10))

# sns.distplot(max_vals_vv,label = 'VV band Max values',color='b')
# # sns.distplot(mean_vals_vv,label = 'VV band Mean values',color = 'g')

# plt.legend()

# plt.title('VV band values Distribution')
# plt.show()

# # band 1value distributions
# plt.figure(figsize=(16,10))

# sns.distplot(max_vals_vh,label = 'VH band Max values',color='b')
# # sns.distplot(mean_vals_vh,label = 'VH band Mean values',color = 'g')


# plt.title('VH band values Distribution')

# plt.legend()
# plt.show()

**Checking to see if the Pipelines work as expected**

In [ ]:
for x,y in S1_dataset_tr.take(1): # take one batch for checking
    print(f'shape of SAR dataset input {x.shape}')

In [ ]:
for x,y in S2_dataset_tr.take(1): # take one batch for checking
    print(f'shape of MultiSpectral dataset input {x.shape}')

In [ ]:
for x,y in RGB_dataset_tr.take(1): # take one batch for checking
    print(f'shape of MultiSpectral dataset input {x.shape}')

#  CNN Models
    CNN models to identify flooding in opotical and SAR images

In [ ]:
def multichannel_cnn(num_channels:int,
                     hidden_units:int, #number of  hidden dense
                     weights = None  # none for random init, use imagenet for imagenet weights
                    ):
    '''model that takes multiple channel as input, instead of using the rgb channels as by default'''


    backbone = tf.keras.applications.resnet_v2.ResNet50V2(
                                            include_top=False,
                                            input_shape = (*CFG.img_size,num_channels),
                                            weights=weights,
                                            pooling = 'avg')


    x = tf.keras.layers.BatchNormalization()(backbone.output)
    x = tf.keras.layers.Dense(hidden_units,
                              activation = 'relu')(x)

    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(rate = 0.2)(x)




    final_out = tf.keras.layers.Dense(2,
                                      activation = 'softmax')(x)


    #make a model
    model = tf.keras.Model(inputs = backbone.input,
                  outputs = final_out)

    return model


# plot train and val acc as  a function of epochs
def plot_history(history,addn_metric=None):
    '''
    Inputs
    history:history object from tensorflow
    add_metric: metric name in the history (like f1_score)'''
    his=pd.DataFrame(history.history)

    if addn_metric:
        plt.subplots(1,3,figsize=(20,6))

        #loss:
        ax1=plt.subplot(1,3,1)
        ax1.plot(range(len(his)),his['loss'],color='g',label='training')
        ax1.plot(range(len(his)),his['val_loss'],color='r',label='validation')
        ax1.set_xlabel('EPOCHS')
        ax1.set_ylabel('LOSS')
        ax1.legend()
        ax1.set_title('Loss Per Epoch')

        #accuracy
        ax2=plt.subplot(1,3,2)
        ax2.plot(range(len(his)),his['accuracy'],color='g',label='training_acc')
        ax2.plot(range(len(his)),his['val_accuracy'],color='r',label='validation_acc')
        ax2.set_xlabel('EPOCHS')
        ax2.set_ylabel('Accuracy')
        ax2.legend()
        ax2.set_title('Accuracy Per Epoch')



        ax3= plt.subplot(1,3,3)
        ax3.plot(range(len(his)),his[f'{addn_metric}'],color='g',label='training')
        ax3.plot(range(len(his)),his[f'val_{addn_metric}'],color='r',label='validation')
        ax3.set_xlabel('EPOCHS')
        ax3.set_ylabel(f'{addn_metric}')
        ax3.legend()
        ax3.set_title(f'{addn_metric} Per Epoch')


    else:
        plt.subplots(1,2,figsize=(20,8))



        #loss:
        ax1=plt.subplot(1,2,1)
        ax1.plot(range(len(his)),his['loss'],color='g',label='training')
        ax1.plot(range(len(his)),his['val_loss'],color='r',label='validation')
        ax1.set_xlabel('EPOCHS')
        ax1.set_ylabel('LOSS')
        ax1.legend()
        ax1.set_title('Loss Per Epoch')

        #accuracy
        ax2=plt.subplot(1,2,2)
        ax2.plot(range(len(his)),his['accuracy'],color='g',label='training_acc')
        ax2.plot(range(len(his)),his['val_accuracy'],color='r',label='validation_acc')
        ax2.set_xlabel('EPOCHS')
        ax2.set_ylabel('Accuracy')
        ax2.legend()
        ax2.set_title('Accuracy Per Epoch')




    plt.show()

# Metrics

In [ ]:


#from https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_score(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)

    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# Building and training the SAR CNN

In [ ]:
SAR_CNN = multichannel_cnn(num_channels = 2,
                           hidden_units = 512, #number of  hidden dense
                          )



SAR_CNN.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
                loss = 'sparse_categorical_crossentropy',
                metrics = ['accuracy',f1_score,recall_m,precision_m]
               )

#check on some data
# SAR_CNN(x)



!mkdir CNN_models

# Defining callbacks

In [ ]:
EPOCHS = 2 if CFG.test_run else 75
# callbacks
#reduce_learning rate
reduce_lr=tf.keras.callbacks.ReduceLROnPlateau(patience=3,
                                                factor=0.75,
                                                min_delta=1e-2,
                                                monitor='val_accuracy',
                                                verbose=1,
                                                mode='max')

#early stopping
early_stopping=tf.keras.callbacks.EarlyStopping(patience=15,
                                              min_delta=1e-3,
                                              monitor='val_accuracy',
                                              restore_best_weights=True,
                                              mode='max')


# exponential decay

def lr_scheduler(epoch, lr):
    '''learning rate scheduler, decays expo after the tenth epoch'''

    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)



learning_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)


callbacks_1= [reduce_lr,early_stopping,learning_scheduler]

**Training on SAR data**

In [ ]:
hist1 = SAR_CNN.fit(S1_dataset_tr,
                    validation_data = S1_dataset_val,
                    epochs = EPOCHS,
                    callbacks = callbacks_1
                   )


#save model
sar_model_path = 'CNN_models/SAR_CNN.h5'
SAR_CNN.save(filepath = 'CNN_models/SAR_CNN.h5')


#plot history
plot_history(hist1,'f1_score')

****Evaluate on validation dataset****

In [ ]:
SAR_CNN.evaluate(S1_dataset_val)

del SAR_CNN;gc.collect()

K.clear_session()

# Model explainablity

    Saliency and Grad Cam. Lets look at what the model is looking at while making htat decision

In [ ]:
!pip install tf_keras_vis -q
import tf_keras_vis

from tf_keras_vis.saliency import Saliency
from tf_keras_vis.gradcam import Gradcam
from tf_keras_vis.utils.scores import CategoricalScore
from tf_keras_vis.utils.model_modifiers import ReplaceToLinear
from tf_keras_vis.gradcam_plus_plus import GradcamPlusPlus


tf_keras_vis.__version__


def model_modifier_function(cloned_model):
    '''modify model activation'''
    cloned_model.layers[-1].activation = tf.keras.activations.linear


def get_saliency(img,
                 score,
                 cnn_model,
                 model_modifier=model_modifier_function):
    #saliency map


    # Create Saliency object.
    saliency = Saliency(cnn_model,
                        model_modifier=model_modifier_function,
                        clone=True)
    #saliency map
    sal_map  = saliency(score,
                        img,
                        smooth_samples=20, # The number of calculating gradients iterations.
                        smooth_noise=0.20) # noise spread level.
    return sal_map

def get_gradcam(img,
                score,
                cnn_model,
                model_modifier=model_modifier_function):

    # Create Gradcam object
    gradcam = Gradcam(cnn_model,
                      model_modifier,
                      clone=True)

    # Generate heatmap with GradCAM
    cam = gradcam(score,
                  img,
                  seek_penultimate_conv_layer=True)

    heatmap = np.uint8(cm.jet(cam[0])[..., :3] * 255)


    return heatmap

def get_gradcam_plus(img,
                    score,
                    model,
                    model_modifier=ReplaceToLinear()):

    # Create GradCAM++ object
    gradcam = GradcamPlusPlus(model,
                          model_modifier=model_modifier,
                          clone=True)

    cam = gradcam(score,
                  img)

    heatmap = np.uint8(cm.jet(cam[0])[..., :3] * 255)

    return heatmap

In [ ]:
SAR_CNN = tf.keras.models.load_model('CNN_models/SAR_CNN.h5',
                                     custom_objects={'f1_score':f1_score,
                                                     'recall_m':recall_m,
                                                     'precision_m':precision_m}
                                    )

In [ ]:


plt.subplots(4,4,figsize=(8*3,8*3))
n = 4
idx= 1


for images,labels in S1_dataset_val.shuffle(buffer_size=5).take(1):
    print(images.shape,labels.shape)

    for i in range(4):
        #get label
        img = images[i]
        lab = int(labels[i].numpy())

        #predict on image
        pred = np.argmax(SAR_CNN.predict(img[tf.newaxis,:,:,:]))
        prd = int(pred.ravel())

        score1 = CategoricalScore(lab)

        plt.subplot(4,4,idx)
        plt.title(f'orignal image ({CFG.class_dict[lab]})')
        plt.axis('off')
        plt.imshow(img[:,:,0])
        idx+=1

        #saliency

        plt.subplot(4,4,idx)
        plt.title(f'predicted {CFG.class_dict[prd]} (saliency map)')
        sal = get_saliency(img,
                           score1,
                           cnn_model = SAR_CNN).squeeze(axis=0)

#         print(sal.shape)
        plt.axis('off')
        plt.imshow(img[:,:,0])
        plt.imshow(sal,alpha=0.25,cmap='jet') #overlay
        idx+=1

        #gradcam
        plt.subplot(4,4,idx)
        gdcam = get_gradcam(img,
                            score1,
                           cnn_model = SAR_CNN)
        plt.imshow(img[:,:,0])
        plt.imshow(gdcam,alpha=0.30,cmap='jet') #overlay
        plt.title(f'predicted {CFG.class_dict[prd]}(gradcam)')
        plt.axis('off')
        idx+=1


        #gradcam ++
        plt.subplot(4,4,idx)
        gdcam_pls = get_gradcam_plus(img,
                                     score1,
                                     model = SAR_CNN)
        plt.imshow(img[:,:,0])
        plt.imshow(gdcam_pls,alpha=0.30,cmap='jet') #overlay
        plt.title(f'predicted {CFG.class_dict[prd]}(gradcam + +)')
        plt.axis('off')
        idx+=1

        if idx>16:
            break

    plt.tight_layout()
    plt.show()

In [ ]:
plt.subplots(4,4,figsize=(8*3,8*3))
n = 4
idx= 1

for images,labels in S1_dataset_val.shuffle(buffer_size=5).take(1):
    print(images.shape,labels.shape)

    for i in range(4):
        #get label
        img = images[i]
        lab = int(labels[i].numpy())

        #predict on image
        pred = np.argmax(SAR_CNN.predict(img[tf.newaxis,:,:,:]))
        score1 = CategoricalScore(lab)
        prd = int(pred.ravel())

        plt.subplot(4,4,idx)
        plt.title(f'orignal image ({CFG.class_dict[lab]})')
        plt.axis('off')
        plt.imshow(img[:,:,1])
        idx+=1
        #saliency
        plt.subplot(4,4,idx)
        plt.title(f'predicted {CFG.class_dict[prd]} (saliency map)')
        sal = get_saliency(img,
                           score1,
                           cnn_model = SAR_CNN).squeeze(axis=0)

        plt.axis('off')
        plt.imshow(img[:,:,1])
        plt.imshow(sal,alpha=0.25,cmap='jet') #overlay
        idx+=1

        #gradcam
        plt.subplot(4,4,idx)
        gdcam = get_gradcam(img,
                            score1,
                           cnn_model = SAR_CNN)
        plt.imshow(img[:,:,1])
        plt.imshow(gdcam,alpha=0.30,cmap='jet') #overlay
        plt.title(f'predicted {CFG.class_dict[prd]}(gradcam)')
        plt.axis('off')
        idx+=1


        #gradcam ++
        plt.subplot(4,4,idx)
        gdcam_pls = get_gradcam_plus(img,
                                     score1,
                                     model = SAR_CNN)
        plt.imshow(img[:,:,1])
        plt.imshow(gdcam_pls,alpha=0.30,cmap='jet') #overlay
        plt.title(f'predicted {CFG.class_dict[prd]}(gradcam + +)')
        plt.axis('off')
        idx+=1

        if idx>16:
            break

    plt.tight_layout()
    plt.show()

In [ ]:
#delete
del SAR_CNN;S1_dataset_val;S1_dataset_tr;gc.collect()

K.clear_session()

# Building and training the RGB - CNN

In [ ]:
RGB_CNN = multichannel_cnn(num_channels = 3,
                           hidden_units = 512, #number of  hidden dense
                           weights = 'imagenet'
                          )


RGB_CNN.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
                loss = 'sparse_categorical_crossentropy',
                metrics = ['accuracy',f1_score,recall_m,precision_m])

In [ ]:
hist2 = RGB_CNN.fit(RGB_dataset_tr,
                    validation_data = RGB_dataset_val,
                    epochs = EPOCHS,
                    callbacks = callbacks_1)

#save model

RGB_CNN.save(filepath = 'CNN_models/RGB_CNN.h5')

plot_history(hist2,'f1_score')

**Checking the GradCAM and saliency Maps for RGB CNN**

In [ ]:


plt.subplots(4,4,figsize=(8*3,8*3))
n = 4
idx= 1


for images,labels in RGB_dataset_val.shuffle(buffer_size=12).take(1):

    for i in range(4):
        #get label
        img = images[i]
        lab = int(labels[i].numpy())


#         print(img.shape,lab.shape)
        score1 = CategoricalScore(lab)



        #predict on image
        prd= np.argmax(RGB_CNN.predict(img[tf.newaxis,:,:,:]))


        plt.subplot(4,4,idx)
        plt.title(f'orignal image ({CFG.class_dict[lab]})')
        plt.axis('off')
        plt.imshow(img)
        idx+=1

        #saliency

        plt.subplot(4,4,idx)
        plt.title(f'predicted {CFG.class_dict[prd]}(saliency map)')
        sal = get_saliency(img,
                           score1,
                           cnn_model = RGB_CNN).squeeze(axis=0)

#         print(sal.shape)
        plt.axis('off')
        plt.imshow(img)
        plt.imshow(sal,alpha=0.45,cmap='jet') #overlay
        idx+=1

        #gradcam
        plt.subplot(4,4,idx)
        gdcam = get_gradcam(img,
                            score1,
                           cnn_model = RGB_CNN)
        plt.imshow(img)
        plt.imshow(gdcam,alpha=0.30,cmap='jet') #overlay
        plt.title(f'predicted {CFG.class_dict[prd]}(gradcam)')
        plt.axis('off')
        idx+=1


        #gradcam ++
        plt.subplot(4,4,idx)
        gdcam_pls = get_gradcam_plus(img,
                                     score1,
                                     model = RGB_CNN)
        plt.imshow(img)
        plt.imshow(gdcam_pls,alpha=0.30,cmap='jet') #overlay
        plt.title(f'predicted {CFG.class_dict[prd]}(gradcam + +)')
        plt.axis('off')
        idx+=1

        if idx>16:
            break

    plt.tight_layout()
    plt.show()


In [ ]:
plt.subplots(4,4,figsize=(8*3,8*3))
n = 4
idx= 1


for images,labels in RGB_dataset_val.shuffle(buffer_size=12).take(1):

    for i in range(4):
        #get label
        img = images[i]
        lab = int(labels[i].numpy())


#         print(img.shape,lab.shape)
        score1 = CategoricalScore(lab)



        #predict on image
        prd= np.argmax(RGB_CNN.predict(img[tf.newaxis,:,:,:]))


        plt.subplot(4,4,idx)
        plt.title(f'orignal image ({CFG.class_dict[lab]})')
        plt.axis('off')
        plt.imshow(img)
        idx+=1

        #saliency

        plt.subplot(4,4,idx)
        plt.title(f'predicted {CFG.class_dict[prd]}(saliency map)')
        sal = get_saliency(img,
                           score1,
                           cnn_model = RGB_CNN).squeeze(axis=0)

#         print(sal.shape)
        plt.axis('off')
        plt.imshow(img)
        plt.imshow(sal,alpha=0.45,cmap='jet') #overlay
        idx+=1

        #gradcam
        plt.subplot(4,4,idx)
        gdcam = get_gradcam(img,
                            score1,
                           cnn_model = RGB_CNN)
        plt.imshow(img)
        plt.imshow(gdcam,alpha=0.30,cmap='jet') #overlay
        plt.title(f'predicted {CFG.class_dict[prd]}(gradcam)')
        plt.axis('off')
        idx+=1


        #gradcam ++
        plt.subplot(4,4,idx)
        gdcam_pls = get_gradcam_plus(img,
                                     score1,
                                     model = RGB_CNN)
        plt.imshow(img)
        plt.imshow(gdcam_pls,alpha=0.30,cmap='jet') #overlay
        plt.title(f'predicted {CFG.class_dict[prd]}(gradcam + +)')
        plt.axis('off')
        idx+=1

        if idx>16:
            break

    plt.tight_layout()
    plt.show()



In [ ]:
#delete
del RGB_CNN;gc.collect()

K.clear_session()

# Building and training the Multispectral - CNN

In [ ]:
multispectral_CNN = multichannel_cnn(num_channels = 12,
                                    hidden_units = 512, #number of  hidden dense
                                    )


#check on some data
# multispectral_CNN(x)


multispectral_CNN.compile(optimizer = tf.keras.optimizers.SGD(learning_rate=0.0001,momentum = 0.0),
                loss = 'sparse_categorical_crossentropy',
                metrics = ['accuracy',f1_score,recall_m,precision_m])

In [ ]:

hist2 = multispectral_CNN.fit(S2_dataset_tr,
                    validation_data = S2_dataset_val,
                    epochs = EPOCHS,
                    callbacks = callbacks_1)

#save model

multispectral_CNN.save(filepath = 'CNN_models/S2_CNN.h5')

plot_history(hist2,'f1_score')

**Checking the decision mechanism for Multispectral model**

In [ ]:


plt.subplots(4,4,figsize=(8*3,8*3))
n = 4
idx= 1


for images,labels in S2_dataset_val.shuffle(buffer_size=12).take(1):
    print(images.shape,labels.shape)

    for i in range(4):
        #get label
        img = images[i]
        lab = int(labels[i].numpy())


#         print(img.shape,lab.shape)
        score1 = CategoricalScore(lab)



        #predict on image
        prd= np.argmax(multispectral_CNN.predict(img[tf.newaxis,:,:,:]))


        plt.subplot(4,4,idx)
        plt.title(f'orignal image ({CFG.class_dict[lab]})')
        plt.axis('off')
        plt.imshow(img[:,:,1:4][:,:,::-1])
        idx+=1

        #saliency

        plt.subplot(4,4,idx)
        plt.title(f'predicted {CFG.class_dict[prd]}(saliency map)')
        sal = get_saliency(img,
                           score1,
                           cnn_model = multispectral_CNN).squeeze(axis=0)

#         print(sal.shape)
        plt.axis('off')
        plt.imshow(img[:,:,1:4][:,:,::-1])
        plt.imshow(sal,alpha=0.45,cmap='jet') #overlay
        idx+=1

        #gradcam
        plt.subplot(4,4,idx)
        gdcam = get_gradcam(img,
                            score1,
                           cnn_model = multispectral_CNN)
        plt.imshow(img[:,:,1:4][:,:,::-1])
        plt.imshow(gdcam,alpha=0.30,cmap='jet') #overlay
        plt.title(f'predicted {CFG.class_dict[prd]}(gradcam)')
        plt.axis('off')
        idx+=1


        #gradcam ++
        plt.subplot(4,4,idx)
        gdcam_pls = get_gradcam_plus(img,
                                     score1,
                                     model = multispectral_CNN)
        plt.imshow(img[:,:,1:4][:,:,::-1])
        plt.imshow(gdcam_pls,alpha=0.30,cmap='jet') #overlay
        plt.title(f'predicted {CFG.class_dict[prd]}(gradcam + +)')
        plt.axis('off')
        idx+=1

        if idx>16:
            break

    plt.tight_layout()
    plt.show()



In [ ]:


plt.subplots(4,4,figsize=(8*3,8*3))
n = 4
idx= 1


for images,labels in S2_dataset_val.shuffle(buffer_size=12).take(1):
    print(images.shape,labels.shape)

    for i in range(4):
        #get label
        img = images[i]
        lab = int(labels[i].numpy())


#         print(img.shape,lab.shape)
        score1 = CategoricalScore(lab)



        #predict on image
        prd= np.argmax(multispectral_CNN.predict(img[tf.newaxis,:,:,:]))


        plt.subplot(4,4,idx)
        plt.title(f'orignal image ({CFG.class_dict[lab]})')
        plt.axis('off')
        plt.imshow(img[:,:,1:4][:,:,::-1])
        idx+=1

        #saliency

        plt.subplot(4,4,idx)
        plt.title(f'predicted {CFG.class_dict[prd]}(saliency map)')
        sal = get_saliency(img,
                           score1,
                           cnn_model = multispectral_CNN).squeeze(axis=0)

#         print(sal.shape)
        plt.axis('off')
        plt.imshow(img[:,:,1:4][:,:,::-1])
        plt.imshow(sal,alpha=0.30,cmap='jet') #overlay
        idx+=1

        #gradcam
        plt.subplot(4,4,idx)
        gdcam = get_gradcam(img,
                            score1,
                           cnn_model = multispectral_CNN)
        plt.imshow(img[:,:,1:4][:,:,::-1])
        plt.imshow(gdcam,alpha=0.30,cmap='jet') #overlay
        plt.title(f'predicted {CFG.class_dict[prd]}(gradcam)')
        plt.axis('off')
        idx+=1


        #gradcam ++
        plt.subplot(4,4,idx)
        gdcam_pls = get_gradcam_plus(img,
                                     score1,
                                     model = multispectral_CNN)
        plt.imshow(img[:,:,1:4][:,:,::-1])
        plt.imshow(gdcam_pls,alpha=0.30,cmap='jet') #overlay
        plt.title(f'predicted {CFG.class_dict[prd]}(gradcam + +)')
        plt.axis('off')
        idx+=1

        if idx>16:
            break

    plt.tight_layout()
    plt.show()


# Model for temporal forecasting

    This notebooks covered the prediction of flooding using satellite images. But this dataset is as time series dataset, which has multiple images of locations before and during the flood. So we will try to cover the temporal aspect of predicting flooding in the next notebook. This can be especially important in some cases where there are water bodies in the image, and hence there needs to be additional signal to distinguish cases of flooding ffrom normal conditions
    
[The temporal forecasting notebook can be found here](https://www.kaggle.com/code/virajkadam/detecting-floods-time-series-prediction)

# References and resources

* https://www.int-arch-photogramm-remote-sens-spatial-inf-sci.net/XLIII-B2-2020/1343/2020/isprs-archives-XLIII-B2-2020-1343-2020.pdf
* https://arxiv.org/pdf/2006.10027v1.pdf#page=2

In [5]:
!pip install sentinelhub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.8/249.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.0/236.0 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.4 MB/s eta 0:00:00


# **maps**

In [20]:
from sentinelhub import SentinelHubRequest, MimeType, CRS, BBox, bbox_to_dimensions, DataCollection

# Define latitude & longitude
latitude = 26.489679   # Change to your location
longitude = 74.550941

# Define bounding box (small area)
bbox = BBox([longitude - 0.01, latitude - 0.01, longitude + 0.01, latitude + 0.01], CRS.WGS84)

# Define image resolution
resolution = 2   # 10m per pixel

# Get dimensions
size = bbox_to_dimensions(bbox, resolution=resolution)

# Create request for Sentinel-2 image
request = SentinelHubRequest(
    evalscript="""
    // Extract RGB image
    function setup() {
        return {
            input: ["B04", "B03", "B02"],
            output: { bands: 3 }
        };
    }
    function evaluatePixel(sample) {
        return [sample.B04, sample.B03, sample.B02];
    }
    """,
    input_data=[SentinelHubRequest.input_data(DataCollection.SENTINEL2_L2A, maxcc=0.1, time_interval=("2024-01-01", "2024-12-31"))],  # Fixed maxcc
    responses=[SentinelHubRequest.output_response("default", MimeType.PNG)],
    bbox=bbox,
    size=size
)

# Get image
image = request.get_data()[0]

# Save image
import imageio
imageio.imwrite("satellite_image.png", image)

print("Image downloaded as satellite_image.png")


Image downloaded as satellite_image.png


In [10]:
from sentinelhub import SHConfig

config = SHConfig()
config.sh_client_id = "7a3098a7-bfcd-4bff-baa4-6104fef14169"
config.sh_client_secret = "bTOS03EAVbfImGlrGqHjMkstr9IJ0aQ1"
config.save()

In [30]:
import pandas as pd
from sentinelhub import SHConfig, BBox, CRS, SentinelHubRequest, DataCollection, MimeType

# Load the dataset
df = pd.read_csv("/home/arshlaan/projext/codathon/flood-risk-in-india/flood_risk_dataset_india.csv")  # Update with the correct file name

# Sort the dataset by flood probability in descending order
df_sorted = df.sort_values(by="Flood Occurred", ascending=False)


# Function to get and save an image for a given latitude and longitude
def download_image(lat, lon, index):
    bbox = BBox(((lon - 0.01, lat - 0.01), (lon + 0.01, lat + 0.01)), crs=CRS.WGS84)

    request = SentinelHubRequest(
        evalscript="""
        // Simple true color script
        function setup() {
            return {
                input: ["B04", "B03", "B02"],
                output: { bands: 3 }
            };
        }
        function evaluatePixel(sample) {
            return [sample.B04, sample.B03, sample.B02];
        }
        """,
        input_data=[
            SentinelHubRequest.input_data(
                DataCollection.SENTINEL2_L2A,
                time_interval=("2024-01-01", "2024-12-31")
            )
        ],
        responses=[SentinelHubRequest.output_response("default", MimeType.PNG)],
        bbox=bbox,
        size=(512, 512),
        config=config
    )

    image = request.get_data()[0]
    file_name = f"flood_risk_{index}.png"
    with open(file_name, "wb") as file:
        file.write(image.tobytes())
    print(f"Saved {file_name}")

# Process and download images for the sorted locations
for idx, row in df_sorted.iterrows():
    download_image(row["Latitude"], row["Longitude"], idx)
    import tensorflow as tf
    import numpy as np
    import cv2
    from PIL import Image
    import rasterio

    # Load models (using compile=False to avoid optimizer warnings)
    model_rgb = tf.keras.models.load_model("/home/arshlaan/projext/codathon/RGB_CNN.h5", compile=False)
    model_s2  = tf.keras.models.load_model("/home/arshlaan/projext/codathon/S2_CNN.h5", compile=False)
    model_sar = tf.keras.models.load_model("/home/arshlaan/projext/codathon/SAR_CNN.h5", compile=False)

    # Upload your image file
    uploaded_img = files.upload()  # When prompted, select your .tif image file
    image_path = list(uploaded_img.keys())[0]
    print("Uploaded image file:", image_path)

    # Load the image using rasterio if it's a .tif
    if image_path.lower().endswith('.tif'):
        with rasterio.open(image_path) as src:
            image_array = src.read()  # shape: (channels, height, width)
        print("Original shape from rasterio:", image_array.shape)

        # Handle channels for S2 model:
        if image_array.shape[0] == 1:
            # Duplicate the single channel 12 times
            image_array = np.repeat(image_array, 12, axis=0)
            print("Duplicated single channel to 12 channels:", image_array.shape)
        elif image_array.shape[0] < 12:
            padding_channels = 12 - image_array.shape[0]
            image_array = np.pad(image_array, ((0, padding_channels), (0,0), (0,0)), mode='constant')
            print("Padded to 12 channels:", image_array.shape)
        elif image_array.shape[0] > 12:
            image_array = image_array[:12, :, :]
            print("Using the first 12 channels:", image_array.shape)

        # Convert from (channels, height, width) to (height, width, channels)
        image_array = np.moveaxis(image_array, 0, -1)
    else:
        # For non-TIFF images, load with PIL and convert to RGB, then pad to 12 channels if needed
        image = Image.open(image_path).convert('RGB')
        image_array = np.array(image)
        if image_array.shape[-1] == 3:
            padding_channels = 12 - 3
            image_array = np.concatenate([image_array, np.zeros((image_array.shape[0], image_array.shape[1], padding_channels), dtype=image_array.dtype)], axis=-1)
            print("Converted non-TIFF image to 12 channels:", image_array.shape)

    print("Shape after processing:", image_array.shape)

    # Resize the image to 256x256 (model's expected input size)
    image_resized = cv2.resize(image_array, (256, 256))
    print("Shape after resizing:", image_resized.shape)

    # Normalize the image (assuming pixel values range 0-255)
    image_normalized = image_resized / 255.0

    # Convert to tensor and add a batch dimension: expected shape (1, 256, 256, 12)
    tensor_image = tf.convert_to_tensor(image_normalized, dtype=tf.float32)
    tensor_image = tf.expand_dims(tensor_image, axis=0)
    print("Final tensor shape:", tensor_image.shape)

    # Prepare inputs for each model:
    # For the RGB model, use the first 3 channels.
    tensor_image_rgb = tensor_image[..., :3]
    # For the S2 model, use all 12 channels.
    tensor_image_s2 = tensor_image
    # For the SAR model, use the first 2 channels.
    tensor_image_sar = tensor_image[..., :2]

    print("RGB input shape:", tensor_image_rgb.shape)
    print("S2 input shape:", tensor_image_s2.shape)
    print("SAR input shape:", tensor_image_sar.shape)

    # Run predictions from each model
    pred_rgb = model_rgb.predict(tensor_image_rgb)
    pred_s2  = model_s2.predict(tensor_image_s2)
    pred_sar = model_sar.predict(tensor_image_sar)

    # Average the predictions for an ensemble output
    avg_pred = np.mean([pred_rgb, pred_s2, pred_sar], axis=0)

    print("RGB Model Prediction:", pred_rgb)
    print("S2 Model Prediction:", pred_s2)
    print("SAR Model Prediction:", pred_sar)
    print("Averaged Prediction:", avg_pred)
    # Assuming avg_pred is [[prob_no_flood, prob_flood]]
    prob_no_flood = avg_pred[0][0] * 100
    prob_flood = avg_pred[0][1] * 100

    # Print predictions in percentage
    print(f"Probability of No Flooding: {prob_no_flood:.2f}%")
    print(f"Probability of Flooding: {prob_flood:.2f}%")

    # Determine and print a message based on which probability is higher
    if prob_flood > prob_no_flood:
        print(f"Flooding is likely with a {prob_flood:.2f}% probability.")
    else:
        print(f"No flooding is detected with a {prob_no_flood:.2f}% probability.")




Saved flood_risk_9984.png


NameError: name 'files' is not defined

In [29]:
import pandas as pd
from sentinelhub import SHConfig, BBox, CRS, SentinelHubRequest, DataCollection, MimeType
import tensorflow as tf
import numpy as np
import cv2
from PIL import Image
import rasterio
import os

# --- Configuration and Dataset Loading ---

# # Set up Sentinel Hub API credentials (update with your actual credentials)
# config = SHConfig()
# config.sh_client_id = "YOUR_CLIENT_ID"
# config.sh_client_secret = "YOUR_CLIENT_SECRET"

# Load the dataset from a local CSV file
dataset_path = "/home/arshlaan/projext/codathon/flood-risk-in-india/flood_risk_dataset_india.csv"
df = pd.read_csv(dataset_path)

# Sort the dataset by flood probability (assumed column "Flood Occurred") in descending order
df_sorted = df.sort_values(by="Flood Occurred", ascending=False)

# Load pre-trained models (update paths as necessary)
model_rgb = tf.keras.models.load_model("/home/arshlaan/projext/codathon/RGB_CNN.h5", compile=False)
model_s2  = tf.keras.models.load_model("/home/arshlaan/projext/codathon/S2_CNN.h5", compile=False)
model_sar = tf.keras.models.load_model("/home/arshlaan/projext/codathon/SAR_CNN.h5", compile=False)

# --- Functions ---

def download_image(lat, lon, index):
    """
    Download a satellite image from Sentinel Hub for the specified latitude and longitude,
    and save it to a designated directory.
    """
    save_dir = "/home/arshlaan/projext/codathon/"
    os.makedirs(save_dir, exist_ok=True)
    
    # Create a bounding box around the coordinate (0.01 degrees offset)
    bbox = BBox(((lon - 0.01, lat - 0.01), (lon + 0.01, lat + 0.01)), crs=CRS.WGS84)
    
    request = SentinelHubRequest(
        evalscript="""
            // Simple true color script
            function setup() {
                return {
                    input: ["B04", "B03", "B02"],
                    output: { bands: 3 }
                };
            }
            function evaluatePixel(sample) {
                return [sample.B04, sample.B03, sample.B02];
            }
        """,
        input_data=[
            SentinelHubRequest.input_data(
                DataCollection.SENTINEL2_L2A,
                time_interval=("2024-01-01", "2024-12-31")
            )
        ],
        responses=[SentinelHubRequest.output_response("default", MimeType.PNG)],
        bbox=bbox,
        size=(512, 512),
        config=config
    )
    
    # Request the image from Sentinel Hub
    image = request.get_data()[0]
    file_name = os.path.join(save_dir, f"flood_risk_{index}.png")
    
    # Save the image file locally
    with open(file_name, "wb") as file:
        file.write(image.tobytes())
    print(f"Saved {file_name}")
    return file_name

def process_and_predict(image_path):
    """
    Process the downloaded image (handling TIFF and non-TIFF cases),
    resize and normalize it, then run predictions using three models.
    """
    print("Processing image:", image_path)
    
    # Check if the file is a TIFF image
    if image_path.lower().endswith('.tif'):
        with rasterio.open(image_path) as src:
            image_array = src.read()  # shape: (channels, height, width)
        print("Original shape from rasterio:", image_array.shape)
        
        # Adjust channels to ensure we have 12 channels for the S2 model:
        if image_array.shape[0] == 1:
            image_array = np.repeat(image_array, 12, axis=0)
            print("Duplicated single channel to 12 channels:", image_array.shape)
        elif image_array.shape[0] < 12:
            padding_channels = 12 - image_array.shape[0]
            image_array = np.pad(image_array, ((0, padding_channels), (0,0), (0,0)), mode='constant')
            print("Padded to 12 channels:", image_array.shape)
        elif image_array.shape[0] > 12:
            image_array = image_array[:12, :, :]
            print("Using the first 12 channels:", image_array.shape)
        
        # Convert from (channels, height, width) to (height, width, channels)
        image_array = np.moveaxis(image_array, 0, -1)
    else:
        # For non-TIFF images, load using PIL
        image = Image.open(image_path).convert('RGB')
        image_array = np.array(image)
        if image_array.shape[-1] == 3:
            padding_channels = 12 - 3
            image_array = np.concatenate([image_array, np.zeros((image_array.shape[0], image_array.shape[1], padding_channels), dtype=image_array.dtype)], axis=-1)
            print("Converted non-TIFF image to 12 channels:", image_array.shape)
    
    print("Shape after processing:", image_array.shape)
    
    # Resize the image to the model's expected input size (256x256)
    image_resized = cv2.resize(image_array, (256, 256))
    print("Shape after resizing:", image_resized.shape)
    
    # Normalize pixel values to [0, 1]
    image_normalized = image_resized / 255.0
    
    # Convert to a TensorFlow tensor and add a batch dimension (1, 256, 256, 12)
    tensor_image = tf.convert_to_tensor(image_normalized, dtype=tf.float32)
    tensor_image = tf.expand_dims(tensor_image, axis=0)
    print("Final tensor shape:", tensor_image.shape)
    
    # Prepare model inputs:
    tensor_image_rgb = tensor_image[..., :3]  # For the RGB model
    tensor_image_s2 = tensor_image           # For the S2 model (all 12 channels)
    tensor_image_sar = tensor_image[..., :2]   # For the SAR model (first 2 channels)
    
    print("RGB input shape:", tensor_image_rgb.shape)
    print("S2 input shape:", tensor_image_s2.shape)
    print("SAR input shape:", tensor_image_sar.shape)
    
    # Run predictions for each model
    pred_rgb = model_rgb.predict(tensor_image_rgb)
    pred_s2  = model_s2.predict(tensor_image_s2)
    pred_sar = model_sar.predict(tensor_image_sar)
    
    # Average the predictions (ensemble)
    avg_pred = np.mean([pred_rgb, pred_s2, pred_sar], axis=0)
    print("RGB Model Prediction:", pred_rgb)
    print("S2 Model Prediction:", pred_s2)
    print("SAR Model Prediction:", pred_sar)
    print("Averaged Prediction:", avg_pred)
    
    # Calculate probabilities (assuming avg_pred format: [[prob_no_flood, prob_flood]])
    prob_no_flood = avg_pred[0][0] * 100
    prob_flood = avg_pred[0][1] * 100
    
    print(f"Probability of No Flooding: {prob_no_flood:.2f}%")
    print(f"Probability of Flooding: {prob_flood:.2f}%")
    
    if prob_flood > prob_no_flood:
        print(f"Flooding is likely with a {prob_flood:.2f}% probability.")
    else:
        print(f"No flooding is detected with a {prob_no_flood:.2f}% probability.")

# --- Main Loop ---
# For each row in the sorted dataset, download the image and process it.
for idx, row in df_sorted.iterrows():
    image_path = download_image(row["Latitude"], row["Longitude"], idx)
    process_and_predict(image_path)


Saved /home/arshlaan/projext/codathon/flood_risk_9984.png
Processing image: /home/arshlaan/projext/codathon/flood_risk_9984.png


UnidentifiedImageError: cannot identify image file '/home/arshlaan/projext/codathon/flood_risk_9984.png'

In [10]:
!pip install opendatasets rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 20.9 MB/s eta 0:00:00


In [4]:
import opendatasets as od
import pandas

od.download(
    "https://www.kaggle.com/datasets/s3programmer/flood-risk-in-india/data")

Dataset URL: https://www.kaggle.com/datasets/s3programmer/flood-risk-in-india


In [1]:
from sentinelhub import SentinelHubRequest, MimeType, CRS, BBox, bbox_to_dimensions, DataCollection

# Define latitude & longitude
latitude = 26.489679   # Change to your location
longitude = 74.550941

# Define bounding box (small area)
bbox = BBox([longitude - 0.01, latitude - 0.01, longitude + 0.01, latitude + 0.01], CRS.WGS84)

# Define image resolution
resolution = 2   # 10m per pixel

# Get dimensions
size = bbox_to_dimensions(bbox, resolution=resolution)

# Create request for Sentinel-2 image
request = SentinelHubRequest(
    evalscript="""
    // Extract RGB image
    function setup() {
        return {
            input: ["B04", "B03", "B02"],
            output: { bands: 3 }
        };
    }
    function evaluatePixel(sample) {
        return [sample.B04, sample.B03, sample.B02];
    }
    """,
    input_data=[SentinelHubRequest.input_data(DataCollection.SENTINEL2_L2A, maxcc=0.1, time_interval=("2024-01-01", "2024-12-31"))],  # Fixed maxcc
    responses=[SentinelHubRequest.output_response("default", MimeType.PNG)],
    bbox=bbox,
    size=size
)

# Get image
image = request.get_data()[0]

# Save image
import imageio
imageio.imwrite("satellite_image.png", image)

print("Image downloaded as satellite_image.png")


/home/arshlaan/projext/codathon/fld/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Image downloaded as satellite_image.png


In [13]:
import pandas as pd
from sentinelhub import SHConfig, BBox, CRS, SentinelHubRequest, DataCollection, MimeType
from sentinelhub import SentinelHubRequest, MimeType, CRS, BBox, bbox_to_dimensions, DataCollection

# Load the dataset
df = pd.read_csv("/home/arshlaan/projext/codathon/flood-risk-in-india/flood_risk_dataset_india.csv")  # Update with the correct file name

# Sort the dataset by flood probability in descending order
df_sorted = df.sort_values(by="Flood Occurred", ascending=False)


    
import tensorflow as tf
import numpy as np
import cv2
from PIL import Image
import rasterio


for index, row in df_sorted.iterrows():
    latitude = row["Latitude"]
    longitude = row["Longitude"]

    bbox = BBox([longitude - 0.01, latitude - 0.01, longitude + 0.01, latitude + 0.01], CRS.WGS84)

    # Define image resolution
    resolution = 2   # 10m per pixel

    # Get dimensions
    size = bbox_to_dimensions(bbox, resolution=resolution)

    # Create request for Sentinel-2 image
    request = SentinelHubRequest(
        evalscript="""
        // Extract RGB image
        function setup() {
            return {
                input: ["B04", "B03", "B02"],
                output: { bands: 3 }
            };
        }
        function evaluatePixel(sample) {
            return [sample.B04, sample.B03, sample.B02];
        }
        """,
        input_data=[SentinelHubRequest.input_data(DataCollection.SENTINEL2_L2A, maxcc=0.1, time_interval=("2024-01-01", "2024-12-31"))],  # Fixed maxcc
        responses=[SentinelHubRequest.output_response("default", MimeType.PNG)],
        bbox=bbox,
        size=size
    )

    # Get image
    image = request.get_data()[0]

    # Save image
    import imageio
    imageio.imwrite("satellite_image.png", image)

    print("Image downloaded as satellite_image.png")
        
    # Load models (using compile=False to avoid optimizer warnings)
    model_rgb = tf.keras.models.load_model("/home/arshlaan/projext/codathon/RGB_CNN.h5", compile=False)
    model_s2  = tf.keras.models.load_model("/home/arshlaan/projext/codathon/S2_CNN.h5", compile=False)
    model_sar = tf.keras.models.load_model("/home/arshlaan/projext/codathon/SAR_CNN.h5", compile=False)

    # Load the image file
    image_path = "/home/arshlaan/projext/codathon/satellite_image.png"  # Replace with the actual path to your image
    print("Processing image file:", image_path)

    # Load the image using rasterio if it's a .tif
    # if image_path.lower().endswith('.tiff'):
        # with rasterio.open(image_path) as src:
        #     image_array = src.read()  # shape: (channels, height, width)
        # print("Original shape from rasterio:", image_array.shape)

        # # Handle channels for S2 model:
        # if image_array.shape[0] == 1:
        #     image_array = np.repeat(image_array, 12, axis=0)
        #     print("Duplicated single channel to 12 channels:", image_array.shape)
        # elif image_array.shape[0] < 12:
        #     padding_channels = 12 - image_array.shape[0]
        #     image_array = np.pad(image_array, ((0, padding_channels), (0,0), (0,0)), mode='constant')
        #     print("Padded to 12 channels:", image_array.shape)
        # elif image_array.shape[0] > 12:
        #     image_array = image_array[:12, :, :]
        #     print("Using the first 12 channels:", image_array.shape)

        # # Convert from (channels, height, width) to (height, width, channels)
        # image_array = np.moveaxis(image_array, 0, -1)
    # if(true):
        # For non-TIFF images, load with PIL and convert to RGB, then pad to 12 channels if needed
    image = Image.open(image_path).convert('RGB')
    image_array = np.array(image)
    if image_array.shape[-1] == 3:
        padding_channels = 12 - 3
        image_array = np.concatenate([image_array, np.zeros((image_array.shape[0], image_array.shape[1], padding_channels), dtype=image_array.dtype)], axis=-1)
        print("Converted non-TIFF image to 12 channels:", image_array.shape)

    print("Shape after processing:", image_array.shape)

    # Resize the image to 256x256 (model's expected input size)
    image_resized = cv2.resize(image_array, (256, 256))
    print("Shape after resizing:", image_resized.shape)

    # Normalize the image (assuming pixel values range 0-255)
    image_normalized = image_resized / 255.0

    # Convert to tensor and add a batch dimension: expected shape (1, 256, 256, 12)
    tensor_image = tf.convert_to_tensor(image_normalized, dtype=tf.float32)
    tensor_image = tf.expand_dims(tensor_image, axis=0)
    print("Final tensor shape:", tensor_image.shape)

    # Prepare inputs for each model:
    tensor_image_rgb = tensor_image[..., :3]  # First 3 channels for RGB model
    tensor_image_s2 = tensor_image  # All 12 channels for S2 model
    tensor_image_sar = tensor_image[..., :2]  # First 2 channels for SAR model

    print("RGB input shape:", tensor_image_rgb.shape)
    print("S2 input shape:", tensor_image_s2.shape)
    print("SAR input shape:", tensor_image_sar.shape)

    # Run predictions from each model
    pred_rgb = model_rgb.predict(tensor_image_rgb)
    pred_s2  = model_s2.predict(tensor_image_s2)
    pred_sar = model_sar.predict(tensor_image_sar)

    # Average the predictions for an ensemble output
    avg_pred = np.mean([pred_rgb, pred_s2, pred_sar], axis=0)

    print("RGB Model Prediction:", pred_rgb)
    print("S2 Model Prediction:", pred_s2)
    print("SAR Model Prediction:", pred_sar)
    print("Averaged Prediction:", avg_pred)

    # Assuming avg_pred is [[prob_no_flood, prob_flood]]
    prob_no_flood = avg_pred[0][0] * 100
    prob_flood = avg_pred[0][1] * 100

    # Print predictions in percentage
    print(f"Probability of No Flooding: {prob_no_flood:.2f}%")
    print(f"Probability of Flooding: {prob_flood:.2f}%")

    # Determine and print a message based on which probability is higher
    if prob_flood > prob_no_flood:
        print(f"Flooding is likely with a {prob_flood:.2f}% probability.")
    else:
        print(f"No flooding is detected with a {prob_no_flood:.2f}% probability.")



Image downloaded as satellite_image.png
Processing image file: /home/arshlaan/projext/codathon/satellite_image.png
Converted non-TIFF image to 12 channels: (1135, 902, 12)
Shape after processing: (1135, 902, 12)
Shape after resizing: (256, 256, 12)
Final tensor shape: (1, 256, 256, 12)
RGB input shape: (1, 256, 256, 3)
S2 input shape: (1, 256, 256, 12)
SAR input shape: (1, 256, 256, 2)
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
RGB Model Prediction: [[3.338135e-05 9.999666e-01]]
S2 Model Prediction: [[1.000000e+00 4.339261e-09]]
SAR Model Prediction: [[1.501649e-28 1.000000e+00]]
Averaged Prediction: [[0.33334446 0.66665554]]
Probability of No Flooding: 33.33%
Probability of Flooding: 66.67%
Flooding is likely with a 66.67% probability.
Image downloaded as satellite_image.png


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f8991d83390>>
Traceback (most recent call last):
  File "/home/arshlaan/projext/codathon/fld/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


Processing image file: /home/arshlaan/projext/codathon/satellite_image.png
Converted non-TIFF image to 12 channels: (1118, 1024, 12)
Shape after processing: (1118, 1024, 12)
Shape after resizing: (256, 256, 12)
Final tensor shape: (1, 256, 256, 12)
RGB input shape: (1, 256, 256, 3)
S2 input shape: (1, 256, 256, 12)
SAR input shape: (1, 256, 256, 2)
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
RGB Model Prediction: [[0.85231286 0.14768717]]
S2 Model Prediction: [[0.9337778  0.06622223]]
SAR Model Prediction: [[3.0293226e-08 1.0000000e+00]]
Averaged Prediction: [[0.59536356 0.40463647]]
Probability of No Flooding: 59.54%
Probability of Flooding: 40.46%
No flooding is detected with a 59.54% probability.
Image downloaded as satellite_image.png
Processing image file: /home/arshlaan/projext/codathon/satellite_image.png
Converted non-TIFF image to 12 channels: (1126, 887, 12)
Shape after processing: (1126, 887, 12)
Shape after res

KeyboardInterrupt: 